In [76]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024')
import importlib
importlib.reload(sys.modules['llm'])
from llm import LM

In [77]:

# Specify the custom cache directory
cache_dir = "/data/rech/huiyuche/huggingface"

# llm = LM(
#     model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
#     tokenizer_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
#     padding_side="left",
#     dtype="bf16",
#     device_map= "auto",
#     use_flash_attention_2=False,
#     access_token=None,
#     cache_dir=cache_dir,
#     accelerator = None
# )
llm = LM(
    model_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
    tokenizer_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
    padding_side="left",
    dtype="bf16",
    device_map= "auto",
    use_flash_attention_2=False,
    access_token=None,
    cache_dir=cache_dir,
    accelerator = None
)

/data/rech/huiyuche/envs/trec_ikat/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.11it/s]


In [63]:
print(type(llm.model))

<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>


In [65]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
outputs, response = llm.hf_llm_generate(
    messages,
    temperature = 0.1,
    top_p = 1,
    max_new_tokens = 256,
    do_sample = True,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [66]:
print(outputs.shape)
print(outputs)
print(response)

torch.Size([1, 75])
tensor([[    1,   733, 16289, 28793,  6526,   460,   368, 28804,   733, 28748,
         16289, 28793,   315,   837,   264,  2475,  3842,  2229,  6202,   486,
         25200,  1650, 16107, 28723,   315,   837,  5682,   298,  8270,  2930,
         28733,  4091,  2245,  2818,   356,   272,  2787,   315,  5556, 28723,
           315,   949, 28742, 28707,   506,   272,  5537,   298,   506,  9388,
         28725, 13855, 28725,   442,   264, 13355, 28723,   315,  2588,   298,
          1316,  4372,  4224, 28725,  8270,  9811,  3036, 28725,   304,  6031,
           395,  4118,  9796, 28723,     2]], device='cuda:0')
I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate creative content, and assist with various tasks.


In [78]:
messages = [
    [{"role": "user", "content": "Who are you?"}]
]
outputs = llm.hf_llm_generate_via_pipline(
    messages,
    temperature = 0.1,
    top_p = 1,
    max_new_tokens = 256,
    do_sample = True,
)

In [79]:
print(outputs)

[" I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate creative content, and assist with various tasks."]


In [11]:
from accelerate import Accelerator
from accelerate import init_empty_weights
from accelerate import load_checkpoint_and_dispatch

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    logging
)


# Specify the custom cache directory
cache_dir = "/data/rech/huiyuche/huggingface"

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct", cache_dir=cache_dir
)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", cache_dir = cache_dir, device_map="auto")

# with init_empty_weights():
#     model = AutoModelForCausalLM()

# model = load_checkpoint_and_dispatch(
#     model,
#     checkpoint=cache_dir + "/models--meta-llama--Meta-Llama-3-8B-Instruct",
#     device_map="auto",
# )
# print(model)
print(model)

accelerator = Accelerator()
print(accelerator.device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.71s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': "I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation and answer questions to the best of my knowledge based on my training data.\n\nI'm a large language model, which means I've been trained on a massive dataset of text from various sources, including books, articles, and websites. This training enables me to understand and generate human-like language, allowing me to engage in conversations, answer questions, and even create text based on a given prompt.\n\nI'm constantly learning and improving my responses based on the interactions I have with users like you. So, feel free to ask me anything, and I'll do my best to provide a helpful and accurate response!"}]}]